# [Elasticsearch]("https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/#delete-items-from-vector-store")

### run docker
`docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.12.1`

In [3]:
from dotenv import load_dotenv
load_dotenv()

import getpass

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## Initialization

In [4]:
from langchain_elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    "langchain-demo", embedding=embeddings, es_url="http://localhost:9200"
)

## Manage vector store
### Add items to vector store

In [13]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['ed2f0b3c-6da1-4f41-bb4f-78d349c61351',
 'be338f5c-2b41-497a-afe7-5e6ec6fee252',
 '3dba30f7-e01b-4033-aa1c-2cb6ca8847e2',
 'c608f496-18d6-4e0b-88f9-3af18e3f8495',
 'ace5ae5b-a712-4b5f-beb3-be46942fba16',
 'fbb176cf-9edb-4258-b1fc-e8ee86b9c00f',
 '84677005-a272-4324-9c36-82c75ada218c',
 '757022c7-1e54-4e29-9ab4-bd169a454328',
 '9d8838ae-fb45-476a-a9d9-f1ac487b1508',
 'c1ae9e80-04bb-4d2d-b85d-9c6694ede68d']

In [11]:
vector_store

Delete items from vector store

In [7]:
vector_store.delete(ids=[uuids[-1]])

True

In [6]:

for id in uuids:
    vector_store.delete(ids=id)

Delete All items

### Query vector store
Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. These examples also show how to use filtering when searching.

#### Query directly
Similarity search  
Performing a simple similarity search with filtering on metadata can be done as follows:

In [14]:
results = vector_store.similarity_search(
    query="LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter=[{"term": {"metadata.source.keyword": "tweet"}}],
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


Similarity search with score  
If you want to execute a similarity search and receive the corresponding scores you can run:

In [16]:
results = vector_store.similarity_search_with_score(
    query="Will it be hot tomorrow",
    k=2,
    filter=[{"term": {"metadata.source.keyword": "news"}}],
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.765917] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.573537] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


#### Query by turning into retriever
You can also transform the vector store into a retriever for easier usage in your chains.

In [19]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2}
)
retriever.invoke("Stealing from the bank is a crime")

[Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.'),
 Document(metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.')]

# FAQ
## Question: Im getting timeout errors when indexing documents into Elasticsearch. How do I fix this?
One possible issue is your documents might take longer to index into Elasticsearch. ElasticsearchStore uses the Elasticsearch bulk API which has a few defaults that you can adjust to reduce the chance of timeout errors.

This is also a good idea when you're using SparseVectorRetrievalStrategy.

The defaults are:

* chunk_size: 500
* max_chunk_bytes: 100MB
To adjust these, you can pass in the `chunk_size` and `max_chunk_bytes` parameters to the ElasticsearchStore `add_texts` method.

In [ ]:
vector_store.add_texts(
    texts,
    bulk_kwargs={
        "chunk_size": 50,
        "max_chunk_bytes": 200000000
    }
)

## Upgrading to ElasticsearchStore
If you're already using Elasticsearch in your langchain based project, you may be using the old implementations: `ElasticVectorSearch` and `ElasticKNNSearch` which are now deprecated. We've introduced a new implementation called `ElasticsearchStore` which is more flexible and easier to use. This notebook will guide you through the process of upgrading to the new implementation.

## What's new?
The new implementation is now one class called `ElasticsearchStore` which can be used for approximate dense vector, exact dense vector, sparse vector (ELSER), BM25 retrieval and hybrid retrieval, via strategies.

## I am using ElasticKNNSearch
Old implementation:

In [21]:

from langchain_community.vectorstores.elastic_vector_search import ElasticKNNSearch

db = ElasticKNNSearch(
  elasticsearch_url="http://localhost:9200",
  index_name="test_index",
  embedding=embedding
)


ImportError: cannot import name 'ElasticKNNSearch' from 'langchain_community.vectorstores.elastic_vector_search' (/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_community/vectorstores/elastic_vector_search.py)

New implementation:

In [23]:

from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy

db = ElasticsearchStore(
  es_url="http://localhost:9200",
  index_name="test_index",
  embedding=embedding,
  # if you use the model_id
  # strategy=DenseVectorStrategy(model_id="test_model")
  # if you use hybrid search
  # strategy=DenseVectorStrategy(hybrid=True)
)


NameError: name 'embedding' is not defined

## I am using ElasticVectorSearch
Old implementation:

In [26]:

from langchain_community.vectorstores.elastic_vector_search import ElasticVectorSearch

db = ElasticVectorSearch(
  elasticsearch_url="http://localhost:9200",
  index_name="test_index",
  embedding=embedding
)


NameError: name 'embedding' is not defined

New implementation:

In [ ]:

from langchain_elasticsearch import ElasticsearchStore, DenseVectorScriptScoreStrategy

db = ElasticsearchStore(
  es_url="http://localhost:9200",
  index_name="test_index",
  embedding=embedding,
  strategy=DenseVectorScriptScoreStrategy()
)


In [ ]:
db.client.indices.delete(
    index="test-metadata, test-elser, test-basic",
    ignore_unavailable=True,
    allow_no_indices=True,
)